Load the __importnb__ to import notebooks.

[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/deathbeds/importnb/master?filepath=readme.ipynb)

## Jupyter Extension

In [1]:
    if __name__ == '__main__':
        %reload_ext importnb
    else: 
        foo = 42
        
    import readme
    assert readme.__file__.endswith('.ipynb')

## Context Manager

In [ ]:
    from importnb import Notebook, reload

    try:  reload(readme)
    except AttributeError: 
        with Notebook(): reload(readme)

## Developer

In [ ]:
    if __name__ == '__main__':
        !jupyter nbconvert --to markdown readme.ipynb